<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex15notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex15A

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．


---
## 拡散モデル
---

---
### VAEの復習 + 階層化



VAE

潜在変数: $\mathbf{z}$， 観測変数: $\mathbf{x}$

- $\mathbf{z}$ を条件としたときの $\mathbf{x}$ の確率分布を $p_{\mathbf{\theta}}(\mathbf{x}|\mathbf{z})$ として，ニューラルネットでモデル化（デコーダ）
- $\mathbf{x}$ を条件としたときの $\mathbf{z}$ の確率分布を $q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})$ として，ニューラルネットでモデル化（エンコーダ）

階層型VAE

潜在変数が $\mathbf{z}_1, \mathbf{z}_2$ の2つある階層型VAE

$\mathbf{z}_2$ から $\mathbf{z}_1$ が生成され，$\mathbf{z}_1$ から $\mathbf{x}$ が生成されると考える．$\mathbf{z}_1$ は $\mathbf{z}_2$ のみで条件付けられ，$\mathbf{x}$ は $\mathbf{z}_1$ のみで条件付けられる（マルコフ性がある）とする．

- $p_{\mathbf{\theta}_2}(\mathbf{z}_1|\mathbf{z}_2), p_{\mathbf{\theta}_1}(\mathbf{x}|\mathbf{z}_1)$ をそれぞれニューラルネットでモデル化（デコーダ）
- $q_{\mathbf{\phi}_1}(\mathbf{z}_1|\mathbf{x}), q_{\mathbf{\phi}_2}(\mathbf{z}_2|\mathbf{z}_1)$ をそれぞれニューラルネットでモデル化（エンコーダ）



一般化して，潜在変数が $\mathbf{z}_1, \mathbf{z}_2, \ldots, \mathbf{z}_T$ の $T$ 個ある階層型VAEも考えられる．

- $p_{\mathbf{\theta}_T}(\mathbf{z}_{T-1}|\mathbf{z}_T), \ldots, p_{\mathbf{\theta}_2}(\mathbf{z}_1|\mathbf{z}_2), p_{\mathbf{\theta}_1}(\mathbf{x}|\mathbf{z}_1)$
- $q_{\mathbf{\phi}_1}(\mathbf{z}_1|\mathbf{x}), q_{\mathbf{\phi}_2}(\mathbf{z}_2|\mathbf{z}_1), \ldots, q_{\mathbf{\phi}_T}(\mathbf{z}_T|\mathbf{z}_{T-1})$

潜在変数を階層化することで，より複雑な分布を表現できると期待されるが，エンコーダデコーダ一つずつから成るVAEと比べると，エンコーダもデコーダも $T$ 個必要となる．
そのため，$T$の大きな階層型VAEを実現するのは，現実には難しい．

---
### 拡散モデルとは



**拡散モデル**（Diffusion Model, 注）には様々なものがあるが，ここではその一つである， **Denoising Diffusion Probabilistic Model** (DDPM)というモデルについて説明する．

※ 注意: ここで説明している「拡散モデル」は，機械学習/深層学習の分野で最近出てきたものである．
一方，数理科学の分野では，様々な現象を記述するモデルとして昔から「拡散モデル」という語が存在している（拡散方程式, etc.）．両者は異なるものであることに注意．

拡散モデルは，階層型VAEを次のように変更したものと考えることができる．

- 観測変数と潜在変数の次元数が等しい
- エンコーダに相当する処理が，パラメータ固定の正規分布に従うノイズを加える計算で表される

このような変更を加えることで，$T=1000$ といった非常に深い階層の潜在変数モデルを実現できる．

拡散モデルでは，観測変数と潜在変数の次元数が等しいことから，どちらも $\mathbf{x}$ という記号で表すことが多い．ここでは，観測変数を $\mathbf{x}_0$ を表し，潜在変数を $\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_T$ と表す．$\mathbf{x}_T$ から $\mathbf{x}_{T-1}$ が生成され，...，$\mathbf{x}_2$ から $\mathbf{x}_1$ が生成され，$\mathbf{x}_1$ から $\mathbf{x}_0$ が生成されると考える．

---
### 拡散過程と逆拡散過程


拡散モデルの計算は，データにノイズを加えていく過程と，加えられたノイズを取り除いていく過程からできていると解釈することができる．
前者を拡散過程といい，後者を逆拡散過程という．

#### 拡散過程

拡散モデルでは，階層型VAEにおける $\mathbf{z}_{t-1}$ から $\mathbf{z}_{t}$ を求める計算が，「時刻 $t-1$ のデータ $\mathbf{x}_{t-1}$ にガウシアンノイズ（正規分布に従うノイズ）を加えて時刻 $t$ のデータ $\mathbf{x}_{t}$ を得る」計算に置き換えられる．
この計算の過程を拡散過程という．

階層型VAEでは， $q_{\mathbf{\phi}_t}(\mathbf{z}_t|\mathbf{z}_{t-1})$ をパラメータ $\mathbf{\phi}_t$ をもつニューラルネット（1つのエンコーダ）でモデル化していたが，拡散モデルでは，$q(\mathbf{x}_t|\mathbf{x}_{t-1})$ はパラメータ固定の正規分布で表される．
典型的には，$q(\mathbf{x}_t|\mathbf{x}_{t-1}) $ が次式に従うと考える．

$$
q(\mathbf{x}_t|\mathbf{x}_{t-1}) = N\left(\mathbf{x}_{t}; \sqrt{1-\beta_t}\mathbf{x}_{t-1}, \beta_tI\right) \qquad (t = 1, 2, \ldots, T)
$$

$\beta_t$ は $0$ 以上 $1$ 以下であらかじめ設定する値である．$\mathbf{x}_{t-1}$ の値が与えられたときに，この正規分布に従う $x_{t}$ のサンプルを得るには， 次のようにすればよい．

$$
\begin{aligned}
\mathbf{\varepsilon} &\sim N(\mathbf{0}, I) \\
\mathbf{x}_t &= \sqrt{1-\beta_t}\mathbf{x}_{t-1} + \sqrt{\beta_t}\mathbf{\varepsilon}
\end{aligned}
$$

例えば，$\beta_t = 0.01$ の場合，$\mathbf{x}_t = 0.995\mathbf{x}_{t-1} + 0.1\mathbf{\varepsilon}$ となり，前の時刻のデータを少しだけ小さくしてから小さなガウシアンノイズを加えることになる．

このように $q(\mathbf{x}_t|\mathbf{x}_{t-1})$ を定めたときに，$T$ を十分大きくし，かつ $\beta_1, \beta_2, \ldots, \beta_T$ がある条件を満たすように定めると，$p(\mathbf{x}_T) \approx N(\mathbf{x}_T; \mathbf{0}, I)$ となることが知られている．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/diffusion.png">

左上の画像に拡散過程を適用していった結果．「Noise: t」の直下の画像は，$\mathbf{x}_t$ に相当する．

#### 逆拡散過程

逆拡散過程では，データ $\mathbf{x}_t$ からノイズを少し取り除いたデータ $\mathbf{x}_{t-1}$ を求める．
この過程の $T$ 回の繰り返しによって，標準正規分布に従う潜在変数 $\mathbf{x}_T$ から観測変数 $\mathbf{x}_0$ が生成される過程をモデル化する．
拡散モデルでは，$\mathbf{x}_{t}$ から $\mathbf{x}_{t-1}$ を求める計算をニューラルネットで表す．ただし，$\mathbf{x}_{T}$ から $\mathbf{x}_{T-1}$ を求める計算，$\mathbf{x}_{T-1}$ から $\mathbf{x}_{T-2}$ を求める計算，...，$\mathbf{x}_{1}$ から $\mathbf{x}_{0}$ を求める計算のそれぞれを異なるニューラルネットで表そうとすると，$T$個のニューラルネット（階層型VAEにおける$T$個のデコーダに相当）が必要となり現実的ではないので，「$\mathbf{x}_t$ と $t$ を入力すると $\mathbf{x}_{t-1}$ の推定値 $\widehat{\mathbf{x}}_{t-1}$を出力するニューラルネット」一つで全てをまかなう．

このニューラルネットを $\textrm{NeuralNet}(\mathbf{x}, t;\mathbf{\theta})$ と表すと，$p_{\mathbf{\theta}}(\mathbf{x}_{t-1}|\mathbf{x}_{t})$ は次のようにモデル化される．

$$
\begin{aligned}
\widehat{\mathbf{x}}_{t-1} &= \textrm{NeuralNet}(\mathbf{x}_t, t;\mathbf{\theta})\\
p_{\mathbf{\theta}}(\mathbf{x}_{t-1}|\mathbf{x}_{t}) &= N(\mathbf{x}_{t-1}; \widehat{\mathbf{x}}_{t-1}, I)
\end{aligned}
$$

このニューラルネットは，入力 $\mathbf{x}_{t}$ と出力 $\widehat{x}_{t-1}$ が同じ次元数なので，オートエンコーダと同じように構成できる．

---
### 拡散モデルの学習


拡散モデルの学習は，VAEと同様に対数尤度の最大化によって行う．
VAEと同様に，対数尤度の代わりにそのELBOを最適化の対象とし，それを近似する，ということで損失関数を導出することができるが，ここではその導出過程の説明は省略する．

逆拡散過程の説明では，拡散モデルのニューラルネット $\textrm{NeuralNet}(\mathbf{x}, t;\mathbf{\theta})$ は，$\mathbf{x}_t$ と $t$ を入力すると $\mathbf{x}_{t-1}$ の推定値 $\widehat{\mathbf{x}}_{t-1}$を出力するものとして定式化した．
しかし，$\mathbf{x}_0$ の推定値を出力させるように変形することができる．
この場合，拡散モデルの学習の手続きは次のようになる：

1. 学習データをひとつ選び，それを $\mathbf{x}_0$ とする
1. $1$ から $T$ までの整数を一様にランダムに選び，それを $t$ とする
1. $\varepsilon \sim N(\mathbf{0}, I)$ を用いて，$\mathbf{x}_0$ にノイズをのせたデータ $\mathbf{x}_{t}$ を求める
1. $\mathbf{x}_t$ と $t$ をニューラルネットに入力して，出力 $\widehat{\mathbf{x}}$ を得る
1. $\mathbf{x}_0$ とニューラルネットの出力との間の二乗誤差（の定数倍） $\gamma_t\Vert \widehat{\mathbf{x}} - \mathbf{x}_0 \Vert^2$ を損失関数としてパラメータを更新する

ノイズをのせたデータ $\mathbf{x}_{t}$ を求める計算と，$\gamma_t$ の式については，説明を省略する．

実用的には，上記のように $\mathbf{x}_0$ を出力のターゲットとするかわりに，$\varepsilon$ をターゲットとする方が良い結果が得られるとの研究報告があり，実装はこの形になっていることが多い．この場合，ノイズののったデータからノイズを取り除いたデータを予測するかわりに，ノイズそのものを予測する形となる．

---
### 拡散モデルによる生成

拡散モデルによるデータ生成の手順は，おおよそ次のようになる（詳細の説明は省略する）．

1. $\mathbf{x}_T \sim N(\mathbf{0}, I)$
1. $t = T, T-1, \ldots, 1$ のそれぞれで次を実行する
    - $\mathbf{x}_t$ と $t$ をニューラルネットに入力し，出力を得る
    - その出力をもとに，$\mathbf{x}_{t-1}$ を求める
1. $\mathbf{x}_0$ を出力する

拡散モデルによるデータの生成では，このようにニューラルネットの出力を求める計算を $T$ 回行う必要がある．
この計算は非常に計算コストが大きいため，様々な改良の工夫が考えられている．

---
### 実装上の工夫

- ニューラルネットの構造としては U-Net を用いることが多い
- 時刻 $t$ の情報は，正弦波等を用いて位置符号化（positional encoding）したベクトルデータの形で入力する
- etc.

---
## Stable Diffusion



---
### Stable Diffusion とは

Stable Diffusion は，画像生成のモデルとしてよく知られたものである（注）．
画像とその内容を表すテキスト情報をペアにしたデータで生成モデルを学習させ，テキストによる指示（プロンプト）に従う画像を生成させることができる．

※注: Stable Diffusion は，生成モデルの名前であるとともに，Stability AI社が提供する生成AIサービスの名前にもなっている．

Stable Diffusion は，VAE, 拡散モデルと，CLIPと呼ばれるニューラルネットを組み合わせたものである．CLIP (Contrastive Language-Image Pre-training)は，画像とその内容を表すテキストのペアを用いて学習したものであり，自然言語のテキストをベクトルデータに変換することができる．

また，Stable Diffusion に用いられる拡散モデルは，データ $\mathbf{x}$ の分布 $p(\mathbf{x})$ をモデル化する代わりに，条件付き分布 $p(\mathbf{x}|y)$ をモデル化する，「条件付き」拡散モデルである．
このような条件付きモデルを用いて，例えば手書き数字画像を $\mathbf{x}$ とし，そのクラスラベルを $y$ として与えて学習させると， 学習後のモデルで $y$ を指定することで，特定の数字画像を生成させることが可能となる．



Stable Diffusion モデルの学習は，次のような手続きとなる：


1. 大量の画像を用いて学習した VAE を用意する
1. 画像とその内容を表すテキストのペアを用いて学習した CLIP を用意する
1. 画像をVAE（のエンコーダ）で符号化した（潜在変数を求めた）ものを $\mathbf{x}$ とし，対応するテキスト情報を CLIPを用いて符号化したものを $y$ として，条件付き分布 $p(\mathbf{x}|y)$ をモデル化する拡散モデルを学習させる．

学習後のモデルを用いて画像を生成する手続きは次のようになる：

1. プロンプトを CLIP で符号化する
1. 符号化したプロンプトを条件として，拡散モデルに画像（の符号）を生成させる
1. 得られた符号をVAEのデコーダに入力して，画像を得る



---
### Stable Diffusion による画像生成の実験

#### 準備

今回の notebook で行う実験の中には，GPU を使える環境でないと動かないものがあります．
次のようにランタイムのタイプを変更しよう．

1. Colab のメニューから「ランタイム」>「ランタイムのタイプを変更」>「CPU」に代えて「T4 GPU」を選択して「保存」
1. ランタイムが初期化されるので，必要なら全てのコードセルを最初から実行し直す


In [ ]:
# diffusers ライブラリのインストール
!pip install diffusers

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

# GPU を使う設定であることを確認
assert torch.cuda.is_available(), 'この実験はGPUが使える環境でないとできません'

# ノイズスケジューラの設定
model_id = 'stabilityai/stable-diffusion-2'
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder='scheduler')

# モデルとそのパラメータの入手
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision='fp16', torch_dtype=torch.float16)
pipe = pipe.to('cuda')

#### 実験

In [ ]:
prompt = 'a photograph of an astronaut riding a horse on mars'
image = pipe(prompt).images[0]
image

引数でいろいろ指定できます．詳しいことが知りたいひとは，リンク先のドキュメントを参照してください．

https://huggingface.co/docs/diffusers/v0.16.0/en/api/pipelines/stable_diffusion/text2img

In [ ]:
prompt = 'high quality, beautiful beach, sunset, (sun), palm tree' # sun の重みを他の 1.1 倍に
neg_prompt = None
image = pipe(prompt, negative_prompt=neg_prompt, guidance_scale=7.5).images[0]
image

In [ ]:
prompt = 'a photo of a man facing his laptop computer and a cat sleeping on the laptop computer'
neg_prompt = 'black cat'
image = pipe(prompt, negative_prompt=neg_prompt, guidance_scale=7.5).images[0]
image

In [ ]:
prompt = 'a figure of mount Fuji erupting with flash of lightning, plume and lava'
neg_prompt = None
image = pipe(prompt, negative_prompt=neg_prompt, guidance_scale=15.0).images[0]
image